# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


Le projet a été préparé par Gabriel Gobeil, associé de recherche à Polytechnique Montréal.

---
# Projet A2022 : Récolte des cerfs de Virginie


La description du projet est disponible à l'adresse suivante : https://www.kaggle.com/t/23b6d8c44f1247c78e19a6a902f804af

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

### Données

Dans un premier temps, vous devrez récupérer les données sur Kaggle. Les fichiers disponibles sont les suivants :
- recolte.csv
- permis.csv
- meteo.csv
- essence.csv
- test.csv

**Déposez ces fichiers dans un dossier nommé *data* dans le répertoire de ce calepin.**

Le fichier *recolte.csv* contient les statistiques de chasse au cerf de Viriginie. On y retrouve le nombre de cerfs mâles adultes récoltés annuellement dans certaines zones de chasse du Québec, listé selon le type d'engin utilisé.

Le fichier *permis.csv* contient des informations sur le nombre de permis de chasse aux cerfs de Virginie alloués aux résidents et non-résidents de 1999 à 2020.

Le fichier *meteo.csv* contient certaines variables météorologiques enregistrées à la station Montréal-Trudeau de 1999 à 2021 :
- Mean Temp (°C)	
- Total Rain (mm)	
- Total Snow (cm)	
- Snow on Grnd (cm)

Le fichier *essence.csv* contient des informations sur le prix de détail moyen par mois de l'essence pour Québec et Montréal de 1999 à 2021. Des renseignements additionnels sur les données sont disponibles à l'adresse suivante : https://www150.statcan.gc.ca/t1/tbl1/fr/cv.action?pid=1810000101

Le fichier *test.csv* contient les zones de chasse pour lesquels vous devez prédire le nombre de cerfs mâles adultes récoltés pour l'année 2021, peu importe l'engin de chasse utilisé. La qualité de vos prédictions sera ensuite évaluée lorsque vous les téléverserez sur Kaggle. Vos prédictions seront comparées à celles des autres équipes de la classe.

### Consignes

- Vous devez constituer une équipe de 3 à 5 personnes.
- Au moins une solution doit être proposée sur Kaggle.
- Utilisez votre identifiant d'équipe pour téléverser vos prédictions sur Kaggle.
- Un seul calepin *.ipynb* par équipe doit être remis. Ce fichier documente et illustre la procédure qui vous a permis de produire vos meilleures prédictions. Ce fichier constitue le rapport final du projet.
- Le langage Julia doit être utilisé.
- Votre démarche doit être rigoureusement justifiée (consultez la grille de correction pour vous orienter).

### Quelques conseils

Votre calepin doit permettre à une personne à l'extérieur de l'équipe de comprendre votre démarche et de reproduire vos résultats. Par exemple, une bonne façon de faire consiste à expliquer dans une cellule de texte la tâche qui est accomplie dans la cellule de code suivante. 

Je vous encourage fortement à faire une analyse exploratoire des données pour développer une meilleure expertise du problème. C'est une étape qui est toujours négligée mais qui est essentielle. C'est avec l'analyse exploratoire que vous viendra des idées d'amélioration, comme par exemple créer de nouvelles variables explicatives.

Vous pouvez utiliser directement tout ce qui se trouve dans les notes de cours sans explication et toutes les librairies utilisées dans le cours (incluant mes fonctions).

Ce calepin contient un modèle très simple de prédiction : on n'utilise qu'une seule variable explicative. Ce sera votre travail d'améliorer ces prédictions avec la méthode et les variables de votre choix.

S'il y a des données manquantes, ce sera à vous de traiter ce problème. La plupart du temps, une méthode simple d'imputation (de remplacement) des données manquantes est suffisante.

Prenez la peine de documenter succinctement les essais infructueux. Ce n'est pas nécessaire de les expliquer en détails, mais c'est important de les mentionner dans la discussion avec une raison possible de leur échec. De cette façon, une personne qui reprendra votre travail dans le futur ne perdra pas de temps à réessayer une méthode infructueuse déjà testée.

Vous pouvez aussi indiquer dans votre rapport les raisons qui vous font croire pourquoi une méthode a moins bien performée de ce qui était attendu. Vous pouvez également mentionner ce que vous auriez pu tenter si vous aviez eu plus de temps ou plus de données. L'idée est de guider l'analyste qui prendrait la relève de votre travail.

Vous êtes limités à deux soumissions par jour UTC par équipe sur Kaggle. Je vous suggère donc de bien tester vos modèles localement et de ne téléverser que vos meilleurs prédictions de la journée.

In [ ]:
import Pkg; Pkg.add("MultivariateStats")

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, DataStructures, Random, StatsBase, LinearAlgebra

---
## 1. Chargement de données

#### Récolte annuelle de cerfs par zones de chasse selon les différents engins

In [ ]:
recolte = CSV.read("data/recolte.csv", DataFrame)
first(recolte, 5)

#### Nombre de permis de chasse alloués annuellement aux résidents et non-résidents

In [ ]:
permis = CSV.read("data/permis.csv", DataFrame)
first(permis, 5)

#### Moyenne mensuelle du prix de détail de l'essence en cents pour Québec et Montréal

In [ ]:
essence = CSV.read("data/essence.csv", DataFrame)
first(essence, 5)

#### Données météorologiques quotidiennes à Montréal-Trudeau entre 1999 et 2021

In [ ]:
meteo = CSV.read("data/meteo.csv", DataFrame)
colnames = ["Date","MeanTemp","TotalRain", "TotalSnow", "SnowOnGrnd"]
rename!(meteo, Symbol.(colnames))
first(meteo, 5)

---
## 2. Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de se familiariser avec les données. C'est une analyse exploratoire sommaire. Je vous encourage fortement à poursuivre cette analyse.

#### 2.1 Quantité de cerfs récoltés annuellement dans toutes les zones pour tous les engins de chasse

In [ ]:
set_default_plot_size(16cm, 10cm)
df = combine(groupby(recolte, :Année), :Cerfs => sum => :Cerfs)
plot(df, x=:Année, y=:Cerfs, Geom.line, Geom.point)

In [ ]:
set_default_plot_size(16cm, 10cm)
plot(permis, x=:Année, y=:Total, Geom.line, Geom.point,
    Guide.ylabel("permis vendus"))

In [ ]:
df2 = innerjoin(df, permis, on = :Année)

plot(df2, x=:Total, y=:Cerfs, Geom.point,
    Guide.xlabel("permis vendus"),
    Guide.ylabel("cerfs récoltés"))

#### 2.2 Quantité de cerfs récoltés annuellement dans les différentes zones de chasse

On doit d'abord additionner le nombre de cerfs récoltés selon les différents engins de chasse.

In [ ]:
zones = groupby(recolte, :Zone)  # On groupe les données par zone

new_df = DataFrame(Année = Int64[], Zone = Int64[], Cerfs= Int64[])  # On initialise un DataFrame vide

for zone in zones  # Pour chaque zone de chasse,
    
    # pour chaque année,
    # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs)  
    
    df2 = DataFrame(Année = df.Année, 
                    Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Cerfs = df.Cerfs) 
    
    append!(new_df, df2)  # On ajoute l'information au DataFrame préinitialisé 
end

sort!(new_df, :Année)
first(new_df, 5)  

In [ ]:
groupby(new_df, :Zone)[1].Zone[1]

In [ ]:
zones = unique(new_df[!, :Zone])
pp = Plot[]
set_default_plot_size(16cm, 10cm)
for i = 1:length(zones)
    p = plot(groupby(new_df, :Zone)[i], x=:Année, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $(groupby(new_df, :Zone)[i].Zone[1])"))
    push!(pp, p)
end

In [ ]:
p = reshape(pp, (19, 1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

#### 2.3 Quantité de données pour chaque zone

Il est pertinent de regarder s'il y a une différence importante entre la quantité de données fournie pour chaque zone

In [ ]:
counter(recolte[!, :Zone])

#### 2.4 Prix de l'essence moyen par année

Cette donnée est fort probablement corélée avec l'année, du à l'inflation, mais cela reste une donnée pertinente

In [ ]:
essence[!, :Année] = Dates.year.(essence[!, :Mois])
# on enlève les années sans données de récolte
essence = essence[essence[!, :Année] .>= 1999, :]
essence = essence[essence[!, :Année] .<= 2020, :]

moy_essence = combine(groupby(essence, :Année), :Québec => mean => :Québec)
moy_mtl = combine(groupby(essence, :Année), :Montréal => mean => :Montréal)  # On groupe les données par zone
moy_essence[!, :Montréal] = moy_mtl.Montréal

set_default_plot_size(16cm, 10cm)
tot_annee = combine(groupby(recolte, :Année), :Cerfs => sum => :Cerfs)
moy_essence[!, :Cerfs] = tot_annee.Cerfs
display(moy_essence)
plot(layer(moy_essence, x=:Québec, y=:Cerfs, Geom.line, Geom.point),
     layer(moy_essence, x=:Montréal, y=:Cerfs, Geom.line, Geom.point))

In [ ]:

set_default_plot_size(16cm, 10cm)
plot(moy_essence, x=:Année, y=:Québec, Geom.line, Geom.point, Guide.xticks(ticks=1999:1:2020),
    Guide.xlabel("Année"), Guide.ylabel("Prix de l'essence (cents/litre)"))

In [ ]:
plot(moy_essence, x=:Année, y=:Cerfs, Geom.line, Geom.point, Guide.xticks(ticks=1999:1:2020),
    Guide.xlabel("Année"), Guide.ylabel("Nombre de cerfs récoltés"))

On voit clairement une relation entre le prix de l'essence et le nombre de cerfs récoltés, cependant ce n'est pas une relation linéaire. Le nombre de Cerfs semble être affecté par des changements soundains du prix.

In [ ]:
# Replace gas price by difference between current and previous year
moy_essence_diff = deepcopy(moy_essence)

for i = 2:size(moy_essence, 1)
    moy_essence_diff[i, :Québec] = (moy_essence[i, :Québec] - moy_essence[i-1, :Québec])
    moy_essence_diff[i, :Montréal] = (moy_essence[i, :Montréal] - moy_essence[i-1, :Montréal])
end

# On enlève la première année, dont on ne peut pas calculer la différence 
moy_essence_diff = moy_essence_diff[2:end, :]
display(moy_essence_diff)

In [ ]:
set_default_plot_size(10cm, 10cm)
plot(
    layer(moy_essence_diff, x=:Année, y=:Québec, Geom.line, Geom.point, Theme(default_color=colorant"green")),
    layer(moy_essence, x=:Année, y=:Québec, Geom.line, Geom.point, Theme(default_color=colorant"red")),
)

##### 2.4.1 Récoltes en fonction des changements du prix de l'essence, par zone

In [ ]:
set_default_plot_size(16cm, 10cm)
zones = groupby(recolte, :Zone)

df_essence_zone = DataFrame(Zone = Int64[], Montréal = Float64[], Cerfs= Int64[])  # On initialise un DataFrame vide

for zone in zones
    
    # pour chaque année,
    # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
    
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs)  
    # remove rows from moy_essence where Année is not in df.Année
    moy_essence_diff_zone = moy_essence_diff[findall(x -> x in df.Année, moy_essence_diff.Année), :]
    df = df[findall(x -> x in moy_essence_diff.Année, df.Année), :]
    df[!, :Montréal] = moy_essence_diff_zone.Montréal
    df2 = DataFrame(Montréal = df.Montréal,
                    Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Cerfs = df.Cerfs) 
    
    append!(df_essence_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
end
display(df_essence_zone)

In [ ]:
zones = unique(df_essence_zone[!, :Zone])
pp = Plot[]
set_default_plot_size(16cm, 10cm)
for i = 1:length(zones)
    p = plot(groupby(df_essence_zone, :Zone)[i], x=:Montréal, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $(groupby(new_df, :Zone)[i].Zone[1])"))
    push!(pp, p)
end

In [ ]:
p = reshape(pp, (19, 1))

set_default_plot_size(20cm, 200cm)
gridstack(p)

On voit parfois une relation linéaire pour certaines zones, il reste à voir si cette relation est significative.

##### 2.4.2 Relation entre changement du prix d'essence et changement du nombre de cerfs

La relation visible en observant les prixs d'essence et le nombre de cerfs pourrait être plus visible en considérant aussi le changement du nombre de cerfs d'une année à l'autre.

In [ ]:
set_default_plot_size(16cm, 10cm)
zones = groupby(recolte, :Zone)

df_essence_zone = DataFrame(Zone = Int64[], Montréal = Float64[], Cerfs= Int64[])  # On initialise un DataFrame vide

for zone in zones
    
    # pour chaque année,
    # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
    
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 
    # replace Cerfs by different in Cerfs from previous year
    for i = 2:size(df, 1)-1
        df[i, :Cerfs] = (df[i, :Cerfs] - df[i-1, :Cerfs])
    end 
    # remove rows from moy_essence where Année is not in df.Année
    moy_essence_diff_zone = moy_essence_diff[findall(x -> x in df.Année, moy_essence_diff.Année), :]
    df = df[findall(x -> x in moy_essence_diff.Année, df.Année), :]
    df[!, :Montréal] = abs.(moy_essence_diff_zone.Montréal)
    df2 = DataFrame(Montréal = df.Montréal,
                    Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Cerfs = df.Cerfs) 
    
    append!(df_essence_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
end
display(df_essence_zone)

In [ ]:
zones = unique(df_essence_zone[!, :Zone])
pp = Plot[]
set_default_plot_size(16cm, 10cm)
for i = 1:length(zones)
    p = plot(groupby(df_essence_zone, :Zone)[i], x=:Montréal, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $(groupby(new_df, :Zone)[i].Zone[1])"))
    push!(pp, p)
end

In [ ]:
p = reshape(pp, (19, 1))

set_default_plot_size(25cm, 200cm)
gridstack(p)

#### 2.5 Prix de l'essence aux alentours des périodes de chasse
Selon https://www.quebec.ca/tourisme-et-loisirs/activites-sportives-et-de-plein-air/chasse-sportive/periodes-limites/cerf-virginie#c128335, les périodes de chasse en 2022 et 2023 semblent être limité de septembre à décembre, les prix de l'essence dans ces périodes pourraient être plus fortement relié au nombre de cerfs. Au lieu de calculer les changements du prix d'essence, il pourrait être plus intéressant d'essayer d'enlever l'effet de l'inflation annuelle, en faisant  un regression linéaire des prix par mois, et par la suite soustraire les valeurs de cette régression aux prix d'essence.

In [ ]:
function GetRowsInPeriod(df)
    df = df[findall(x -> x ∉ [4,5,6,7,8,9], Dates.month.(df.Date)), :]
    # # not allowed?
    # # remove rows in october not in first 20 days
    # df_october = df[findall(x -> x == 10, Dates.month.(df.Date)), :]
    # df_october = df_october[findall(x -> x <= 20, Dates.day.(df_october.Date)), :]

    # # remove rows in november not between day 5 and 21
    # df_november = df[findall(x -> x == 11, Dates.month.(df.Date)), :]
    # df_november = df_november[findall(x -> x <= 21, Dates.day.(df_november.Date)), :]
    # df_november = df_november[findall(x -> x >= 5, Dates.day.(df_november.Date)), :]

    # return vcat(df_october, df_november)
    return df
end

##### 2.5.1 Prix de l'essence ajusté pour l'inflation
En effectuant une régression linéaire sur les prix d'essence par mois, on peut ajuster les prix pour l'inflation. On peut ensuite soustraire les valeurs de cette régression aux prix d'essence pour enlever l'effet de l'inflation.

In [ ]:
# convert date to number
function date_to_number(date)
    return Dates.year(date) + Dates.month(date)/12 + Dates.day(date)/365
end

# remove rows no in october to december
essence_chasse = deepcopy(essence)
rename!(essence_chasse, Symbol("Mois") => :Date)
essence_chasse = GetRowsInPeriod(essence_chasse)

essence_annuelle = deepcopy(essence)
essence_annuelle[!, :Date] = date_to_number.(deepcopy(essence)[!, :Mois])
essence_model = lm(@formula(Montréal ~ Date), essence_annuelle)

In [ ]:
# substract value from regression from prices in Montreal 
temp_df = DataFrame(Date = date_to_number.(essence_chasse[!, :Date]))
essence_chasse[!, :Montréal_adjusted] = essence_chasse[!, :Montréal] - predict(essence_model, temp_df)
display(essence_chasse)



In [ ]:
B_0 = coef(essence_model)[1]
B_1 = coef(essence_model)[2]
set_default_plot_size(25cm, 20cm)
plot(
    layer(essence_annuelle, x=:Date, y=:Montréal, Geom.line, Geom.point, Theme(default_color=colorant"green"),
          intercept=[B_0], slope=[B_1], Geom.abline(style=:dash, color="red"))
    )

In [ ]:
# on enlève les années sans données de récolte
essence_chasse = essence_chasse[essence_chasse[!, :Année] .>= 1999, :]
essence_chasse = essence_chasse[essence_chasse[!, :Année] .<= 2020, :]

moy_essence_chasse = combine(groupby(essence_chasse, :Année), :Montréal_adjusted => mean => :Montréal_adjusted)

set_default_plot_size(16cm, 10cm)
tot_annee = combine(groupby(recolte, :Année), :Cerfs => sum => :Cerfs)
tot_annee = tot_annee[findall(x -> x in moy_essence.Année, tot_annee.Année), :]
moy_essence_chasse[!, :Cerfs] = tot_annee.Cerfs
set_default_plot_size(25cm, 10cm)
plot(layer(moy_essence_chasse, x=:Montréal_adjusted, y=:Cerfs, Geom.line, Geom.point))



##### 2.5.2 Prix de l'essence ajusté avec le prix des mois précédents
On pourrais penser qu'un changement soudain au prix de l'essence pendant la période de chasse pourrait avoir un effet sur le nombre de cerfs récoltés. On pourrait donc soustraire le prix moyen de l'essence pendant la période de chasse au prix de l'essence des mois précédents.

In [ ]:

nbr_mois = 1

essence = CSV.read("data/essence.csv", DataFrame)
rename!(essence, Symbol("Mois") => :Date)
essence[!, :Année] = Dates.year.(essence[!, :Date])
# on enlève les années sans données de récolte
essence = essence[essence[!, :Année] .>= 1999, :]
essence = essence[essence[!, :Année] .<= 2020, :]

essence_chasse = GetRowsInPeriod(deepcopy(essence))
essence_avant_chasse = deepcopy(essence[findall(x -> x in (9-nbr_mois+1):9, Dates.month.(essence.Date)), :])

moy_essence = combine(groupby(essence_chasse, :Année), :Montréal => mean => :Montréal, :Québec => mean => :Québec)
moy_essence_avant_chasse = combine(groupby(essence_avant_chasse, :Année), :Montréal => mean => :Montréal, :Québec => mean => :Québec)

moy_essence[!, :Montréal] = moy_essence[!, :Montréal] - moy_essence_avant_chasse.Montréal
moy_essence[!, :Québec] = moy_essence[!, :Québec] - moy_essence_avant_chasse[!, :Québec]

set_default_plot_size(16cm, 10cm)
tot_annee = combine(groupby(recolte, :Année), :Cerfs => sum => :Cerfs)
moy_essence[!, :Cerfs] = tot_annee.Cerfs
# moy_essence[!, :Date] = Dates.year.(moy_essence[!, :Date])
display(moy_essence)


In [ ]:
zones = groupby(recolte, :Zone)

df_essence_zone = DataFrame(Zone = Int64[], Essence = Float64[], Cerfs = Int64[])
for zone in zones
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

    moy_essence_temp = moy_essence[findall(x -> x in df.Année, moy_essence.Année), :]
    
    df2 = DataFrame(Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Essence = moy_essence_temp.Montréal,
                    Cerfs = df.Cerfs) 
    append!(df_essence_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
end

In [ ]:
zones = unique(df_essence_zone[!, :Zone])
pp = Plot[]
set_default_plot_size(16cm, 10cm)
for i = 1:length(zones)
    p = plot(groupby(df_essence_zone, :Zone)[i], x=:Essence, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $(groupby(new_df, :Zone)[i].Zone[1])"))
    push!(pp, p)
end

p = reshape(pp, (19, 1))

set_default_plot_size(25cm, 200cm)
gridstack(p)

On ne trouve toujours pas de relation linéaire entre le prix de l'essence et le nombre de cerfs récoltés.

#### 2.6 Relation entre température et nombre de cerfs récoltés

##### 2.6.1 Température moyenne en période de chasse

In [ ]:
function getTempMoy()
    meteo_moy = GetRowsInPeriod(deepcopy(meteo))
    meteo_moy = meteo_moy[.!ismissing.(meteo_moy.MeanTemp), :]
    meteo_moy[!, :Date] = Dates.year.(meteo_moy[!, :Date])
    # on enlève les années sans données de récolte
    meteo_moy = meteo_moy[meteo_moy[!, :Date] .>= 1999, :]
    meteo_moy = meteo_moy[meteo_moy[!, :Date] .<= 2020, :]

    meteo_moy = combine(groupby(meteo_moy, :Date), :MeanTemp => mean => :MeanTemp)

    # min_temp = minimum(meteo_moy.MeanTemp)
    # meteo_moy[!, :MeanTemp] = meteo_moy[!, :MeanTemp] .- (min_temp + 0.01)
    return meteo_moy
end

In [ ]:
meteo_moy = getTempMoy()
zones = groupby(recolte, :Zone)

df_temp_moy_zone = DataFrame(Zone = Int64[], Temp = Float64[], Cerfs = Int64[])  # On initialise un DataFrame vide

for zone in zones
    
    # pour chaque année,
    # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
    
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

    # on a pas les données de température pour 2013, on enlève l'année
    df = df[df[!, :Année] .!= 2013, :]
    meteo_moy_temp = meteo_moy[findall(x -> x in df.Année, meteo_moy.Date), :]
    df2 = DataFrame(Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Temp = meteo_moy_temp.MeanTemp,
                    Cerfs = df.Cerfs) 
    
    append!(df_temp_moy_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
end

In [ ]:
zones = unique(df_temp_moy_zone[!, :Zone])
pp = Plot[]
set_default_plot_size(16cm, 10cm)
for i = 1:length(zones)
    p = plot(groupby(df_temp_moy_zone, :Zone)[i], x=:Temp, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $(groupby(new_df, :Zone)[i].Zone[1])"))
    push!(pp, p)
end

p = reshape(pp, (19, 1))

set_default_plot_size(25cm, 200cm)
gridstack(p)

##### 2.6.2 Nombre de jours avec une température moyenne en dessous d'un certain seuil l'an dernier

Un hiver plus froid pourrait avoir un effet sur le nombre de cerfs récoltés l'année suivante.

In [ ]:
function count_cold_days(limite_temp, getDataTest = false, power = 1)
    meteo_count = deepcopy(meteo)
    meteo_count = meteo_count[.!ismissing.(meteo_count.MeanTemp), :]
    meteo_count[!, :Date] = Dates.year.(meteo_count[!, :Date])


    meteo_count[!, :JourFroid] .= false

    meteo_count[meteo_count[!, :MeanTemp] .< limite_temp, :JourFroid] .= true
    meteo_count = combine(groupby(meteo_count, :Date), :JourFroid => sum => :JourFroid)
    
    meteo_count[!, :JourFroid] .= Int64.(ceil.(meteo_count[!, :JourFroid]  .^ power))

    df_temp_count_zone = DataFrame(Année = Int64[], Zone = Int64[], CountJoursFroid = Float64[], Cerfs = Int64[])  # On initialise un DataFrame vide

    if getDataTest
        meteo_count = meteo_count[end-1, :]
        return DataFrame(CountJoursFroid = meteo_count.JourFroid) 
    else
        meteo_count = meteo_count[meteo_count[!, :Date] .>= 1998, :]
        meteo_count = meteo_count[meteo_count[!, :Date] .<= 2020, :]
    end

    zones = groupby(recolte, :Zone)


    for zone in zones
        
        # pour chaque année,
        # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
        
        df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

        # on a pas les données de cerfs pour 1999 et 2014
        df = df[df[!, :Année] .!= 1999, :]
        df = df[df[!, :Année] .!= 2014, :]
        meteo_count_new = meteo_count[findall(x -> (x+1) in df.Année, meteo_count.Date), :]
        df = df[findall(x -> (x-1) in meteo_count_new.Date, df.Année), :]
        df2 = DataFrame(Année = df.Année,
                        Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                        CountJoursFroid = meteo_count_new.JourFroid,
                        Cerfs = df.Cerfs) 
        
        append!(df_temp_count_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
    end
    return df_temp_count_zone
end

In [ ]:
pp = Plot[]

for i = 1:18
    df = count_cold_days(5)
    p = plot(groupby(df, :Zone)[i], x=:CountJoursFroid, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $i - Limite température = 5"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 500cm)
gridstack(p)

#### 2.7 Relation entre neige au sol et Cerfs récoltés
De façon similaire avec la température, on pourrait essayer de compter le nombre de jours où il y avait beaucoup de neige au sol, ce qui pourrait avoir un effet sur le nombre de cerfs récoltés l'année suivante.

In [ ]:
function count_snow_grnd_days(limite_neige, getDataTest = false, power = 1)
    meteo_count = deepcopy(meteo)
    meteo_count[ismissing.(meteo_count.SnowOnGrnd), :SnowOnGrnd] .= 0
    meteo_count[!, :Date] = Dates.year.(meteo_count[!, :Date])

    meteo_count[!, :JourNeigeGrnd] .= false

    meteo_count[meteo_count[!, :SnowOnGrnd] .> limite_neige, :JourNeigeGrnd] .= true
    meteo_count = combine(groupby(meteo_count, :Date), :JourNeigeGrnd => sum => :JourNeigeGrnd)

    meteo_count[!, :JourNeigeGrnd] .= Int64.(ceil.(meteo_count[!, :JourNeigeGrnd]  .^ power))

    df_neige_grnd_count_zone = DataFrame(Année = Int64[], Zone = Int64[], CountJoursNeigeGrnd = Float64[], Cerfs = Int64[])  # On initialise un DataFrame vide

    if getDataTest
        meteo_count = meteo_count[end-1, :]
        return DataFrame(CountJoursNeigeGrnd = meteo_count.JourNeigeGrnd) 
    else
        meteo_count = meteo_count[meteo_count[!, :Date] .>= 1998, :]
        meteo_count = meteo_count[meteo_count[!, :Date] .<= 2020, :]
    end

    zones = groupby(recolte, :Zone)

    for zone in zones
        
        # pour chaque année,
        # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
        
        df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

        # on a pas les données de cerfs pour 1999 et 2014
        df = df[df[!, :Année] .!= 1999, :]
        df = df[df[!, :Année] .!= 2014, :]
        meteo_count_new = meteo_count[findall(x -> (x+1) in df.Année, meteo_count.Date), :]
        df = df[findall(x -> (x-1) in meteo_count_new.Date, df.Année), :]
        df2 = DataFrame(Année = df.Année,
                        Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                        CountJoursNeigeGrnd = meteo_count_new.JourNeigeGrnd,
                        Cerfs = df.Cerfs) 
        
        append!(df_neige_grnd_count_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
    end
    return df_neige_grnd_count_zone
end

In [ ]:
pp = Plot[]
limite_neige_grnd = 8
for i = 1:18
    df = count_snow_grnd_days(limite_neige_grnd)
    p = plot(groupby(df, :Zone)[i], x=:CountJoursNeigeGrnd, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $i - Limite température = 5"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 500cm)
gridstack(p)

#### 2.8 Relation entre neige tombée et Cerfs récoltés
De la même façon on pourrais essayer de compter le nombre de jours où il y a eu beaucoup de neige tombée, ce qui pourrait avoir un effet sur le nombre de cerfs récoltés l'année suivante.

In [ ]:
function count_snow_fall_days(limite_neige, getDataTest = false, power = 1)
    meteo_count = deepcopy(meteo)
    meteo_count[ismissing.(meteo_count.TotalSnow), :TotalSnow] .= 0
    # meteo_count = meteo_count[.!ismissing.(meteo_count.TotalSnow), :]
    meteo_count[!, :Date] = Dates.year.(meteo_count[!, :Date])

    meteo_count[!, :JourNeigeFall] .= false

    meteo_count[meteo_count[!, :TotalSnow] .> limite_neige, :JourNeigeFall] .= true
    meteo_count = combine(groupby(meteo_count, :Date), :JourNeigeFall => sum => :JourNeigeFall)

    meteo_count[!, :JourNeigeFall] .= Int64.(ceil.(meteo_count[!, :JourNeigeFall]  .^ power))

    df_neige_fall_count_zone = DataFrame(Année = Int64[], Zone = Int64[], CountJoursNeigeFall = Float64[], Cerfs = Int64[])  # On initialise un DataFrame vide

    if getDataTest
        meteo_count = meteo_count[end-1, :]
        return DataFrame(CountJoursNeigeFall = meteo_count.JourNeigeFall) 
    else
        meteo_count = meteo_count[meteo_count[!, :Date] .>= 1998, :]
        meteo_count = meteo_count[meteo_count[!, :Date] .<= 2020, :]
    end

    zones = groupby(recolte, :Zone)

    for zone in zones
        
        # pour chaque année,
        # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
        
        df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

        # on a pas les données de cerfs pour 1999 et 2014
        df = df[df[!, :Année] .!= 1999, :]
        df = df[df[!, :Année] .!= 2014, :]
        meteo_count_new = meteo_count[findall(x -> (x+1) in df.Année, meteo_count.Date), :]
        df = df[findall(x -> (x-1) in meteo_count_new.Date, df.Année), :]
        df2 = DataFrame(Année = df.Année,
                        Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                        CountJoursNeigeFall = meteo_count_new.JourNeigeFall,
                        Cerfs = df.Cerfs) 
        
        append!(df_neige_fall_count_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
    end
    return df_neige_fall_count_zone
end

On peut en premier chercher la limite de neige qui semble avoir la meilleure relation avec le nombre de cerfs récoltés en essayant différentes valeurs.

In [ ]:
## Assez lent car on génère 400 graphes
#=
pp = Plot[]
zone_test = 6
for i = 0:0.1:40
    df = count_snow_fall_days(i)
    p = plot(groupby(df, :Zone)[zone_test], x=:CountJoursNeigeFall, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $zone_test - Limite température = $i"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 5000cm)
gridstack(p)
=#

On voit qu'à une valeur limite d'environ 2cm, il y a une assez bonne relation, du moins pour la zone 6. On peut voir si il y a aussi une relation pour les autres zones.

In [ ]:

pp = Plot[]
limite_neige = 2

for i = 1:18
    df = count_snow_fall_days(limite_neige)
    p = plot(groupby(df, :Zone)[i], x=:CountJoursNeigeFall, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $i - Limite neige = $limite_neige"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 500cm)
gridstack(p)

Il semble en effet avoir une relation entre le nombre de cerfs récoltés et le nombre de jours avec beaucoup de neige tombée l'année précédente. Comme on s'y attendait, des hivers difficile pour les cerfs ont un effet sur le nombre de cerfs récoltés l'année suivante.

#### 2.9 Relation entre quantité de pluie tombée et Cerfs récoltés
On fait la même chose pour la pluie

In [ ]:
function count_rainy_days(limite_pluie, getDataTest = false)
    meteo_count = deepcopy(meteo)
    meteo_count[ismissing.(meteo_count.TotalRain), :TotalRain] .= 0
    # meteo_count = meteo_count[.!ismissing.(meteo_count.TotalSnow), :]
    meteo_count[!, :Date] = Dates.year.(meteo_count[!, :Date])

    meteo_count[!, :JourPluie] .= false

    meteo_count[meteo_count[!, :TotalRain] .> limite_pluie, :JourPluie] .= true
    meteo_count = combine(groupby(meteo_count, :Date), :JourPluie => sum => :JourPluie)

    df_pluie_zone = DataFrame(Année = Int64[], Zone = Int64[], CountJourPluie = Float64[], Cerfs = Int64[])  # On initialise un DataFrame vide

    if getDataTest
        meteo_count = meteo_count[end-1, :]
        display(meteo_count)
        return DataFrame(CountJoursNeigeFall = meteo_count.JourPluie) 
    else
        meteo_count = meteo_count[meteo_count[!, :Date] .>= 1998, :]
        meteo_count = meteo_count[meteo_count[!, :Date] .<= 2020, :]
    end

    zones = groupby(recolte, :Zone)

    for zone in zones
        
        # pour chaque année,
        # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
        
        df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs) 

        # on a pas les données de cerfs pour 1999 et 2014
        df = df[df[!, :Année] .!= 1999, :]
        df = df[df[!, :Année] .!= 2014, :]
        meteo_count_new = meteo_count[findall(x -> (x+1) in df.Année, meteo_count.Date), :]
        df = df[findall(x -> (x-1) in meteo_count_new.Date, df.Année), :]
        df2 = DataFrame(Année = df.Année,
                        Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                        CountJourPluie = meteo_count_new.JourPluie,
                        Cerfs = df.Cerfs) 
        
        append!(df_pluie_zone, df2)  # On ajoute l'information au DataFrame préinitialisé 
    end
    return df_pluie_zone
end

In [ ]:
## Assez lent car on génère 400 graphes
#=
pp = Plot[]
zone_test = 6
for i = 0:0.1:40
    df = count_rainy_days(i)
    p = plot(groupby(df, :Zone)[zone_test], x=:CountJourPluie, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $zone_test - Limite pluie = $i"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 5000cm)
gridstack(p)
=#

De la même façon on trouve une relation linéaire avec une limite d'environ 13.5 cm de pluie.

In [ ]:
pp = Plot[]
zone_test = 6
limite_pluie = 13.5
for i = 1:18
    df = count_rainy_days(limite_pluie)
    p = plot(groupby(df, :Zone)[i], x=:CountJourPluie, y=:Cerfs, Geom.point, Geom.line, Guide.title("Zone $i - Limite pluie = $limite_pluie"))
    push!(pp, p)
end

p = reshape(pp, (length(pp), 1))

set_default_plot_size(20cm, 500cm)
gridstack(p)

La relation ne semble pas être vraie pour les autres zones.

 #### 2.10 Relation entre permis 

---
## 3. Ajustement d'un modèle de régression linéaire simple

Pour cet exemple simple, on n'utilise que l'année comme variable explicative afin d'ajuster un modèle de régression linéaire pour chaque zone de chasse.

#### 3.1. Ajustement des modèles 

In [ ]:
models = DataFrame(Zone = Int64[], β̂₀ = Float64[], β̂₁ = Float64[])  # On initialise le DataFrame vide

zones = groupby(new_df, :Zone)  # On groupe les données par zone
for zone in zones  # Pour chaque zone de chasse,

    y = zone.Cerfs
    n = length(y)
    
    x₁ = zone.Année  # On considère l'année seulement
    X = hcat(ones(n), x₁)

    β̂₀, β̂₁=  X'X\X'y  # Estimation des coefficients de régression
    
    push!(models, [zone.Zone[1], β̂₀, β̂₁])  # On ajoute ça au DataFame
end

models

#### 3.2. Validation graphique

Pour avoir un aperçu de la qualité de la regression, on ajoute la droite de régression.

In [ ]:
function f(x, i) 
    models[i, :β̂₀] + x * models[i, :β̂₁]
end


In [ ]:
zones = unique(new_df[!, :Zone])

pp = Plot[]

set_default_plot_size(10cm, 6cm)
for i = 1:length(zones)
    p = plot(
            layer(groupby(new_df, :Zone)[i], x=:Année, y=:Cerfs, Geom.point, Geom.line,
            intercept=[models[i, :β̂₀]], slope=[models[i, :β̂₁]], Geom.abline(style=:dash, color="red")),
            layer(x -> f(x, i), 1999, 2021))
    push!(pp, p)
end

In [ ]:
# Pour afficher les 4 premières zones 

p = reshape(pp, (19,1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

---
## 4. Estimation du nombre de cerfs mâles adultes récoltés par zone de chasse en 2021 

On utilise le modèle simple de la section précédente pour estimer le nombre de cerfs récoltés pour chacune des lignes de l'ensemble de test.

#### 4.1 Chargement des données de l'ensemble de test

In [ ]:
test = CSV.read("data/test.csv", DataFrame);
display(test)

#### 4.2 Estimation du nombre de cerfs récoltés pour chacune des ligne de l'ensemble de test.

In [ ]:
models = models[in.(models.Zone, Ref(test.Zone)), :];  # On récupère seulement les modèles pour les zones à prévoir

In [ ]:
ŷ = [Int64(ceil(f(2021, i))) for i in 1:size(test,1)];

#### 4.3 Préparation du fichier des préditions pour téléverser sur Kaggle

Le fichier *benchmark_predictions.csv* généré peut être téléversé sur Kaggle. Il est composé d'une colonne de zones (Zone) et d'une colonne du nombre de cerfs récoltés (Cerfs).

In [ ]:
prediction = DataFrame(Zone = test.Zone, Cerfs = ŷ)

CSV.write("benchmark_predictions.csv", prediction)

---
## 5. Utilisation de variables explicatives identifiées dans la section 2

### 5.1 Modèle avec année, température moyenne pendant la période de chasse et nombre de jours froids l'an passé

#### 5.1.1 Ajustement du modèle

In [ ]:
temp_limit = 5
meteo_moy = getTempMoy()
new_zones = DataFrame(Zone = Int64[], Année = Int64[], Cerfs = Int64[], CountJoursFroid = Int64[], MoyTemp = Float64[])
meteo_count_zone = count_cold_days(temp_limit)
zones = groupby(meteo_count_zone, :Zone)
for (i, zone) in enumerate(zones)
    zone = zone[zone[!, :Année] .!= 2013, :]
    meteo_moy_temp = meteo_moy[findall(x -> x in zone.Année, meteo_moy.Date), :]
    zone[!, :MoyTemp] .= meteo_moy_temp.MeanTemp
    append!(new_zones, zone)
end

new_zones = groupby(new_zones, :Zone)

In [ ]:
models = Any[]

for zone in new_zones
    model = lm(@formula(Cerfs ~ Année + MoyTemp + CountJoursFroid), zone)
    push!(models, model)
end

#### 5.1.2 Validation graphique

In [ ]:
for (i, zone) in enumerate(new_zones)
    zone[!, :Prediction] .= 0
    for j = 1:size(zone, 1)
        data = DataFrame(Année = zone.Année[j], MoyTemp = zone.MoyTemp[j], CountJoursFroid = zone.CountJoursFroid[j])
        ŷ = Int64.(ceil.(predict(models[i], data)[1]))
        zone[j, :Prediction] = Int64(ceil(ŷ[1]))
    end
end


In [ ]:
pp = Plot[]

for i = 1:length(new_zones)
    p = plot(
            layer(new_zones[i], x=:Année, y=:Cerfs, Geom.point, Geom.line, Theme(default_color=colorant"green")),
            layer(new_zones[i], x=:Année, y=:Prediction, Geom.point, Geom.line, Theme(default_color=colorant"red")))
    push!(pp, p)
end

p = reshape(pp, (19,1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

In [ ]:
# Calculate average squared error for all predictions
function mse(predictions, actual)
    sum((predictions .- actual).^2) / length(predictions)
end

# Calculate average squared error for all predictions


#### 5.1.3 Estimation du nombre de cerfs récoltés en 2021

In [ ]:
prediction = DataFrame(Zone = Int64[], Cerfs = Int64[])

cold_days_2020 = count_cold_days(temp_limit, true)[1, :].CountJoursFroid

meteo_moy = deepcopy(meteo)
meteo_moy = GetRowsInPeriod(deepcopy(meteo))
meteo_moy = meteo_moy[.!ismissing.(meteo_moy.MeanTemp), :]
meteo_moy[!, :Date] = Dates.year.(meteo_moy[!, :Date])
meteo_moy = meteo_moy[meteo_moy[!, :Date] .== 2021, :]

moy_temp_2021 = combine(groupby(meteo_moy, :Date), :MeanTemp => mean => :MeanTemp)[1, :].MeanTemp

for model in models
    data = DataFrame(Année = 2021, MoyTemp = moy_temp_2021, CountJoursFroid = cold_days_2020)
    pred = max(Int64(ceil(predict(model, data)[1])), 0)
    push!(prediction, [test.Zone[1], pred])
end
prediction = prediction[prediction[!, :Zone] .!= 21, :]
CSV.write("Charles_Année-moyTemp-Jours5C.csv", prediction)

### 5.2 Modèle avec année, température moyenne pendant la période de chasse, nombre de jours froids l'an passé et cerfs récoltés l'année précédente

#### 5.2.1 Ajustement du modèle

In [ ]:
temp_limit = 5
meteo_moy = getTempMoy()
new_zones = DataFrame(Zone = Int64[], Année = Int64[], Cerfs = Int64[], CountJoursFroid = Int64[], MoyTemp = Float64[], RecolteAnneePassee = Int64[])
meteo_count_zone = count_cold_days(temp_limit)
zones = groupby(meteo_count_zone, :Zone)

zones_cerfs = groupby(recolte, :Zone)
zones_cerfs = [zones_cerfs[i][zones_cerfs[i][!, :Année] .!= 2013, :] for i in 1:length(zones_cerfs)]

for (i, zone) in enumerate(zones)
    zone = zone[zone[!, :Année] .!= 2013, :]
    meteo_moy_temp = meteo_moy[findall(x -> x in zone.Année, meteo_moy.Date), :]
    zone[!, :MoyTemp] .= meteo_moy_temp.MeanTemp
    
    zone_cerfs = combine(groupby(zones_cerfs[i], :Année), :Cerfs => sum => :Cerfs)
    zone_cerfs[!, :Année] = zone_cerfs[!, :Année] .+ 1
    zone_cerfs = zone_cerfs[findall(x -> x in meteo_moy_temp.Date, zone_cerfs.Année), :]
    zone = zone[findall(x -> x in zone_cerfs.Année, zone.Année), :]
    zone[!, :RecolteAnneePassee] .= zone_cerfs.Cerfs
    append!(new_zones, zone)
end

new_zones = groupby(new_zones, :Zone)

In [ ]:
models = Any[]

for zone in new_zones
    model = lm(@formula(Cerfs ~ Année + MoyTemp + CountJoursFroid + RecolteAnneePassee), zone)
    push!(models, model)
end

#### 5.2.2 Validation graphique

In [ ]:
for (i, zone) in enumerate(new_zones)
    zone[!, :Prediction] .= 0
    for j = 1:size(zone, 1)
        data = DataFrame(Année = zone.Année[j], MoyTemp = zone.MoyTemp[j], CountJoursFroid = zone.CountJoursFroid[j], RecolteAnneePassee = zone.RecolteAnneePassee[j])
        ŷ = predict(models[i], data)
        zone[j, :Prediction] = Int64(ceil(ŷ[1]))
    end
end


In [ ]:
pp = Plot[]

for i = 1:length(new_zones)
    p = plot(
            layer(new_zones[i], x=:Année, y=:Cerfs, Geom.point, Geom.line, Theme(default_color=colorant"green")),
            layer(new_zones[i], x=:Année, y=:Prediction, Geom.point, Geom.line, Theme(default_color=colorant"red")))
    push!(pp, p)
end

p = reshape(pp, (19,1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

In [ ]:
# Calculate average squared error for all predictions
function mse(predictions, actual)
    sum((predictions .- actual).^2) / length(predictions)
end

# Calculate average squared error for all predictions
score = 0
for zone in new_zones
    score += mse(zone.Prediction, zone.Cerfs)
end
score = sqrt(score / length(new_zones))

#### 5.2.3 Estimation du nombre de cerfs récoltés en 2021

In [ ]:
prediction = DataFrame(Zone = Int64[], Cerfs = Int64[])

cold_days_2020 = count_cold_days(temp_limit, true)[1, :].CountJoursFroid

meteo_moy = deepcopy(meteo)
meteo_moy = GetRowsInPeriod(deepcopy(meteo))
meteo_moy = meteo_moy[.!ismissing.(meteo_moy.MeanTemp), :]
meteo_moy[!, :Date] = Dates.year.(meteo_moy[!, :Date])
meteo_moy = meteo_moy[meteo_moy[!, :Date] .== 2021, :]

moy_temp_2021 = combine(groupby(meteo_moy, :Date), :MeanTemp => mean => :MeanTemp)[1, :].MeanTemp

zone_cerfs = deepcopy(recolte)
zone_cerfs = groupby(recolte, :Zone)
for (i, model) in enumerate(models)
    recoltes_temp = combine(groupby(zone_cerfs[i], :Année), :Cerfs => sum => :Cerfs)
    sort!(recoltes_temp, :Année)
    recolte_annee_passee = recoltes_temp[end, :].Cerfs
    data = DataFrame(Année = 2021, MoyTemp = moy_temp_2021, CountJoursFroid = cold_days_2020, RecolteAnneePassee = recolte_annee_passee)
    pred = max(Int64(ceil(predict(model, data)[1])), 0)
    push!(prediction, [zone_cerfs[i].Zone[1], pred])
end
prediction = prediction[prediction[!, :Zone] .!= 21, :]
CSV.write("Charles_Année-moyTemp-Jours5C-recolte.csv", prediction)

### 5.3 Ajout de quantité de jours avec beaucoup de neige au sol l'an passé

##### 5.3.1 Ajustement du modèle

In [ ]:
temp_limit = 5
limite_neige_grnd = 8
meteo_moy = getTempMoy()
new_zones = DataFrame(Zone = Int64[], Année = Int64[], Cerfs = Int64[], CountJoursFroid = Int64[], MoyTemp = Float64[], RecolteAnneePassee = Int64[], CountJoursNeigeGrnd = Int64[])
meteo_count_zone = count_cold_days(temp_limit)
neige_grnd = count_snow_grnd_days(limite_neige_grnd)


meteo_count_zone[!, :CountJoursNeigeGrnd] .= neige_grnd[!, :CountJoursNeigeGrnd]

zones = groupby(meteo_count_zone, :Zone)

zones_cerfs = groupby(recolte, :Zone)
zones_cerfs = [zones_cerfs[i][zones_cerfs[i][!, :Année] .!= 2013, :] for i in 1:length(zones_cerfs)]

for (i, zone) in enumerate(zones)
    zone = zone[zone[!, :Année] .!= 2013, :]
    meteo_moy_temp = meteo_moy[findall(x -> x in zone.Année, meteo_moy.Date), :]
    zone[!, :MoyTemp] .= meteo_moy_temp.MeanTemp
    
    zone_cerfs = combine(groupby(zones_cerfs[i], :Année), :Cerfs => sum => :Cerfs)
    zone_cerfs[!, :Année] = zone_cerfs[!, :Année] .+ 1
    zone_cerfs = zone_cerfs[findall(x -> x in meteo_moy_temp.Date, zone_cerfs.Année), :]
    zone = zone[findall(x -> x in zone_cerfs.Année, zone.Année), :]
    zone[!, :RecolteAnneePassee] .= zone_cerfs.Cerfs
    append!(new_zones, zone)
end

new_zones = groupby(new_zones, :Zone)

In [ ]:
models = Any[]

for zone in new_zones
    model = lm(@formula(Cerfs ~ Année + MoyTemp + CountJoursFroid + RecolteAnneePassee + CountJoursNeigeGrnd), zone)
    push!(models, model)
end

##### 5.3.2 Validation graphique

In [ ]:
for (i, zone) in enumerate(new_zones)
    zone[!, :Prediction] .= 0
    for j = 1:size(zone, 1)
        data = DataFrame(Année = zone[j, :Année], MoyTemp = zone[j, :MoyTemp], CountJoursFroid = zone[j, :CountJoursFroid], RecolteAnneePassee = zone[j, :RecolteAnneePassee], CountJoursNeigeGrnd = zone[j, :CountJoursNeigeGrnd])
        ŷ = Int64(ceil(predict(models[i], data)[1]))
        zone[j, :Prediction] = Int64(ceil(ŷ[1]))
    end
end

In [ ]:
pp = Plot[]

for i = 1:length(new_zones)
    p = plot(
            layer(new_zones[i], x=:Année, y=:Cerfs, Geom.point, Geom.line, Theme(default_color=colorant"green")),
            layer(new_zones[i], x=:Année, y=:Prediction, Geom.point, Geom.line, Theme(default_color=colorant"red")))
    push!(pp, p)
end

p = reshape(pp, (19,1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

In [ ]:
# Calculate average squared error for all predictions
function mse(predictions, actual)
    sum((predictions .- actual).^2) / length(predictions)
end

# Calculate average squared error for all predictions
score = 0
for zone in new_zones
    score += mse(zone.Prediction, zone.Cerfs)
end
score = sqrt(score / length(new_zones))

### 5.4 Ajout de quantité de jours avec beaucoup de neige tombée l'an passé

##### 5.4.1 Ajustement du modèle

In [ ]:
temp_limit = 5
limite_neige_grnd = 8
limite_neige_fall = 2
meteo_moy = getTempMoy()
new_zones = DataFrame(Zone = Int64[], Année = Int64[], Cerfs = Int64[], CountJoursFroid = Int64[], MoyTemp = Float64[], RecolteAnneePassee = Int64[], CountJoursNeigeGrnd = Int64[], CountJoursNeigeFall = Int64[])
meteo_count_zone = count_cold_days(temp_limit, false, 1)
neige_grnd = count_snow_grnd_days(limite_neige_grnd, false, 1)
snow_fall = count_snow_fall_days(limite_neige_fall, false, 1)

meteo_count_zone[!, :CountJoursNeigeGrnd] .= neige_grnd[!, :CountJoursNeigeGrnd]
meteo_count_zone[!, :CountJoursNeigeFall] .= snow_fall[!, :CountJoursNeigeFall]

zones = groupby(meteo_count_zone, :Zone)

zones_cerfs = groupby(recolte, :Zone)
zones_cerfs = [zones_cerfs[i][zones_cerfs[i][!, :Année] .!= 2013, :] for i in 1:length(zones_cerfs)]

for (i, zone) in enumerate(zones)
    zone = zone[zone[!, :Année] .!= 2013, :]
    meteo_moy_temp = meteo_moy[findall(x -> x in zone.Année, meteo_moy.Date), :]
    zone[!, :MoyTemp] .= meteo_moy_temp.MeanTemp
    
    zone_cerfs = combine(groupby(zones_cerfs[i], :Année), :Cerfs => sum => :Cerfs)
    zone_cerfs[!, :Année] = zone_cerfs[!, :Année] .+ 1
    zone_cerfs = zone_cerfs[findall(x -> x in meteo_moy_temp.Date, zone_cerfs.Année), :]
    zone = zone[findall(x -> x in zone_cerfs.Année, zone.Année), :]
    zone[!, :RecolteAnneePassee] .= zone_cerfs.Cerfs
    append!(new_zones, zone)
end


train_size = 0.90
Random.seed!(11231)
ids = collect(axes(new_zones, 1))
shuffle!(ids)
train_zones = new_zones[ids[1:round(Int, train_size * length(ids))], :]
valid_zones = new_zones[ids[round(Int, train_size * length(ids)):end], :]
train_zones = groupby(train_zones, :Zone)
valid_zones = groupby(valid_zones, :Zone)


In [ ]:
models = Any[]

for zone in train_zones
    model = lm(@formula(Cerfs ~ Année + MoyTemp + CountJoursFroid + RecolteAnneePassee + CountJoursNeigeGrnd + CountJoursNeigeFall), zone)
    push!(models, model)
end

##### 5.4.3 Validation graphique

In [ ]:
grouped_zones = groupby(deepcopy(new_zones), :Zone)
for (i, zone) in enumerate(grouped_zones)
    zone[!, :Prediction] .= 0
    for j = 1:size(zone, 1)
        data = DataFrame(Année = zone.Année[j], MoyTemp = zone.MoyTemp[j], CountJoursFroid = zone.CountJoursFroid[j], RecolteAnneePassee = zone.RecolteAnneePassee[j], CountJoursNeigeGrnd = zone.CountJoursNeigeGrnd[j], CountJoursNeigeFall = zone.CountJoursNeigeFall[j])
        ŷ = predict(models[i], data)
        zone[j, :Prediction] = Int64(ceil(ŷ[1]))
    end
end

In [ ]:
pp = Plot[]

for i = 1:length(grouped_zones)
    p = plot(
            layer(grouped_zones[i], x=:Année, y=:Cerfs, Geom.point, Geom.line, Theme(default_color=colorant"green")),
            layer(grouped_zones[i], x=:Année, y=:Prediction, Geom.point, Geom.line, Theme(default_color=colorant"red")))
    push!(pp, p)
end

p = reshape(pp, (19,1))

set_default_plot_size(25cm, 100cm)
gridstack(p)

In [ ]:
function getModelIndexFromZone(zone)

    to_return = 0
    for i = 1:nrow(test)
        if test[i, :Zone] == zone
            to_return = i
            break
        end
    end
    if zone == 21
        return 17
    elseif zone >= 21
        return to_return + 1
    else
        return to_return
    end
end

In [ ]:
function getScore(df, models)
    df_temp = deepcopy(df)
    for (i, zone) in enumerate(df_temp)
        zone[!, :Prediction] .= 0
        for j = 1:size(zone, 1)
            data = DataFrame(Année = zone.Année[j], MoyTemp = zone.MoyTemp[j], CountJoursFroid = zone.CountJoursFroid[j], RecolteAnneePassee = zone.RecolteAnneePassee[j], CountJoursNeigeGrnd = zone.CountJoursNeigeGrnd[j], CountJoursNeigeFall = zone.CountJoursNeigeFall[j])
            ŷ = predict(models[getModelIndexFromZone(zone[!, :Zone][1])], data)
            zone[j, :Prediction] = Int64(ceil(ŷ[1]))
        end
    end

    score = 0
    for zone in df_temp
        score += mse(zone.Prediction, zone.Cerfs)
    end
    return sqrt(score / length(valid_zones))
end

In [ ]:
println("train score: ", getScore(train_zones, models))
println("valid score: ", getScore(valid_zones, models))
println("total score: ", getScore(grouped_zones, models))

#### 5.5 Regression Ridge

In [ ]:
models = DataFrame(lambda = Float64[], Zone = Int64[], model = Any[])
standard_fits = DataFrame(Zone = Int64[], dt = Any[])
valid_performance = DataFrame(Zone = Int64[], lambda = Float64[], rmse = Float64[])

display(train_zones[1][:, 2:end])
for lambda = 0:0.1:50
    
    # Avec seulement 2 points pour la zone 28, on ne peut pas faire de régression linéaire
    for (i, zone) in enumerate(train_zones[1:end-1])
        train = zone[:, 2:end]
        dt = StatsBase.fit(StatsBase.ZScoreTransform, Matrix{Float64}(train), dims=1)
        push!(standard_fits, [zone[1, 1], dt])
        trans_train = StatsBase.transform(dt, Matrix{Float64}(train))

        X = trans_train[:, 1:end .!= 2] 
        y = trans_train[:, 2]

        B = (X'X + lambda*I)\X'y

        push!(models, [lambda, zone[1, 1], B])
    end

    for (i, zone) in enumerate(valid_zones)
        valid = zone[:, 2:end]
        dt = standard_fits[standard_fits[!, :Zone] .== zone[1, :Zone], :].dt[1]
        trans_valid = StatsBase.transform(dt, Matrix{Float64}(valid))
        X_valid = trans_valid[:, 1:end .!=2]
        y_valid = trans_valid[:, 2]

        B = models[models[!, :Zone] .== zone[1, :Zone], :].model[end]
        y_pred = X_valid * B

        rmse = StatsBase.rmsd(y_pred, y_valid)
        push!(valid_performance, [zone[1, :Zone], lambda, rmse])
    end
end

In [ ]:
valid_performance_zone = groupby(valid_performance, :Zone)
best_lambdas = DataFrame(Zone = Int64[], lambda = Float64[])
for zone in valid_performance_zone
    push!(best_lambdas, [zone[1, :Zone], zone.lambda[argmin(zone.rmse)]])
end
display(best_lambdas)

#### 5.5.2 Prédiction avec modèle Ridge 

In [ ]:
prediction_scaled = DataFrame(Zone = Int64[], Cerfs = Float64[])
prediction_unscaled = DataFrame(Zone = Int64[], Cerfs = Int64[])

cold_days_2020 = count_cold_days(temp_limit, true)[1, :].CountJoursFroid

meteo_moy = deepcopy(meteo)
meteo_moy = GetRowsInPeriod(deepcopy(meteo))
meteo_moy = meteo_moy[.!ismissing.(meteo_moy.MeanTemp), :]
meteo_moy[!, :Date] = Dates.year.(meteo_moy[!, :Date])
meteo_moy = meteo_moy[meteo_moy[!, :Date] .== 2021, :]

neige_grnd = count_snow_grnd_days(limite_neige_grnd, true, 1)[1, :].CountJoursNeigeGrnd
snow_fall = count_snow_fall_days(limite_neige_fall, true, 1)[1, :].CountJoursNeigeFall

moy_temp_2021 = combine(groupby(meteo_moy, :Date), :MeanTemp => mean => :MeanTemp)[1, :].MeanTemp

zone_cerfs = deepcopy(recolte)
zone_cerfs = zone_cerfs[zone_cerfs[!, :Zone] .!= 21, :]
zone_cerfs = zone_cerfs[zone_cerfs[!, :Zone] .!= 28, :]
zone_cerfs = groupby(zone_cerfs, :Zone)

for (i, zone) in enumerate(zone_cerfs)
    recoltes_temp = combine(groupby(zone_cerfs[i], :Année), :Cerfs => sum => :Cerfs)
    sort!(recoltes_temp, :Année)
    recolte_annee_passee = recoltes_temp[end, :].Cerfs

    data = DataFrame(Année = 2021,  Cerfs = 0, CountJoursFroid = cold_days_2020,MoyTemp = moy_temp_2021, RecolteAnneePassee = recolte_annee_passee, CountJoursNeigeGrnd = neige_grnd, CountJoursNeigeFall = snow_fall)
    data = Matrix{Float64}(data)

    B = models[models[!, :lambda] .== (best_lambdas[best_lambdas[!, :Zone] .== zone.Zone[1], :].lambda[1]), :].model[1]
    dt = standard_fits[standard_fits[!, :Zone] .== zone.Zone[1], :].dt[1]
    data = StatsBase.transform(dt, data)
    data = data[:, 1:end .!= 2]
    display(data)
    pred = (data * B)[1]
    push!(prediction_scaled, [zone.Zone[1], pred])
end
# prediction = prediction[prediction[!, :Zone] .!= 21, :]

temp_df = deepcopy(new_zones)
temp_df[!,:Cerfs] = convert.(Float64, temp_df[!,:Cerfs])
temp_df[1:nrow(prediction_scaled), :Cerfs] = prediction_scaled[!, :Cerfs]
println(prediction_scaled[!, :Cerfs])
for i = 1:nrow(prediction_scaled)
    dt = standard_fits[standard_fits[!, :Zone] .== prediction_scaled.Zone[i], :].dt[1]
    line_to_reconstruct = transpose(Matrix(temp_df)[i, 2:end])
    # display(line_to_reconstruct)
    push!(prediction_unscaled, [prediction.Zone[i], max(Int64(ceil(StatsBase.reconstruct(dt, line_to_reconstruct)[2])), 0) ])
    println(line_to_reconstruct)
end
CSV.write("Charles_Année-moyTemp-Jours5C-recolte-Ridge.csv", prediction_unscaled)